In [3]:
import pandas as pd


# 1) EXPLORACIÓN DE LOS DATOS 

En mi computaor la carga de los datos completos se demora 20min, por esto voy a disminuir el tamaño de los dataframes para poder ilustrar el proceso

In [20]:
books = pd.read_csv('books_data.csv', nrows=100000) #cargamos solamente las primeras 100000 filas de los dataframes
ratings = pd.read_csv('Books_rating.csv', nrows=100000)

In [26]:
books.head()


,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


In [27]:
ratings.head()

,Id,Title,User_id,review/score,review/text
0,1882931173,Its Only Art If Its Well Hung!,AVCGYZL8FQQTD,4.0,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,A30TK6U7DNS82R,5.0,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,A3UH4UZ4RSVO82,5.0,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,A2MVUWT453QH61,4.0,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,A22X4XUPKF66MR,4.0,Philip Nel - Dr. Seuss: American IconThis is b...


De los dos dataframes anteriores identificamos que las variables más importantes para realizar el modelo de recomendación son: el número de 

## 1.1 Exploremos la cantidad de valores faltantes

Las variables que nos interesan del dataframe rating son: la identificación del lubri, el título del libro, la idntificación del usuario, la calificación, 

In [22]:
ratings=ratings[['Id', 'Title','User_id', 'review/score', 'review/text']]

In [17]:
print(books.isnull().sum())
print('\n')
print(ratings.isnull().sum())

Title                1
description      32851
authors          15695
image            25234
previewLink      12164
publisher        36283
publishedDate    12858
infoLink         12164
categories       20175
ratingsCount     76556
dtype: int64


Id                        0
Title                     1
Price                 87018
User_id               19216
profileName           19220
review/helpfulness        0
review/score              0
review/time               0
review/summary            2
review/text               0
dtype: int64


In [12]:
books= books.fillna('')
books

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],
4,"Nation Dance: Religion, Identity and Cultural ...",,['Edward Long'],,http://books.google.nl/books?id=399SPgAACAAJ&d...,,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,,
...,...,...,...,...,...,...,...,...,...,...
99995,Six years in the Malay jungle,,['Carveth Wells'],http://books.google.com/books/content?id=ToMcA...,http://books.google.com/books?id=ToMcAAAAMAAJ&...,,1925,https://play.google.com/store/books/details?id...,['Ethnology'],
99996,Timescapes: Stories of Time Travel F&c; Sheets,Short stories based on the theme of time-trave...,['Peter Haining'],http://books.google.com/books/content?id=t7pTA...,http://books.google.com/books?id=t7pTAAAACAAJ&...,,1997,http://books.google.com/books?id=t7pTAAAACAAJ&...,"['Science fiction, American']",
99997,Holy Vulnerability,Life can seemingly be fine on the surface. But...,['Kellye Fabian'],http://books.google.com/books/content?id=oAspE...,http://books.google.com/books?id=oAspEAAAQBAJ&...,NavPress,2021-07-06,http://books.google.com/books?id=oAspEAAAQBAJ&...,['Religion'],
99998,From Good Goddess to Vestal Virgins: Sex and C...,The role of women in Roman culture and society...,['Ariadne Staples'],http://books.google.com/books/content?id=cCqmz...,http://books.google.com/books?id=cCqmzwXnHhcC&...,Routledge,2013-02-01,https://play.google.com/store/books/details?id...,['History'],


### Vamos a manejar los valores faltantes y los duplicados

# Modelo de recomendación

# 